[Reference](https://pub.towardsai.net/time-series-forecasting-building-and-deploying-models-continued-deee0588b28c)

# ARIMA


In [1]:
# Import label data from profile.txt file in UCI repository  and name columns
label = pd.read_csv(".hyddata\\profile.txt",delim_whitespace=True,header=None)
label.columns = ['cooler_condition', 'valve_condition', 'pump_leak', 'hydraulic_accumulator', 'stable_flag']
# Split label into training, then import and fit ARIMA model
label_train, label_test = label.loc[:1000,:], label.loc[1001:,:]
from statsmodels.tsa.arima.model import ARIMA
hydmodel = ARIMA(label_train['hydraulic_accumulator'],order=(100,0,0))
hydmodel_fit = hydmodel.fit()

In [2]:
resids = DataFrame(hydmodel_fit.resid)
resids.plot(kind='kde')

In [3]:
resids.describe()

In [5]:
preds = hydmodel_fit.forecast(100)

from sklearn.metrics import mean_absolute_error
mean_absolute_error(label_test_hyd[:100].values,preds)

In [6]:
leakpreds = leakmodel_fit.forecast(100)
mean_absolute_error(label_test_leak[:100].values,leakpreds)

# Auto ML


In [7]:
%pip install h2o
import h2o
from h2o.automl import H2OAutoML
h2o.init(ip="127.0.0.1", port="8080")

In [8]:
# Creating combined dataframe of labels and features, then splitting into training and testing sets.
combined_extracted = extracted_features.join(label)
combined_extracted.reset_index(inplace=True)
train_extracted = combined_extracted.iloc[:1100,:]
test_extracted = combined_extracted.iloc[1101:,:]
# Below: creating H2Oframes for use in modeling
hf_train_extracted = h2o.H2OFrame(train_extracted)
hf_test_extracted = h2o.H2OFrame(test_extracted)
# Below: declare independent (X) and dependent (y) variables
y = 'hydraulic_accumulator'
X = hf_train_extracted.columns
X.remove(y)

In [9]:
aml_search_hyd = H2OAutoML(max_runtime_secs = 600)
aml_search_hyd.train(x = X, 
          y = y,
          training_frame = hf_train_extracted,
          leaderboard_frame = hf_test_extracted)
aml_search_hyd.leaderboard
# Code above declares and fits H2O Auto ML model on training set, and asks for a leaderboard of top performers using test set.
# Repeating earlier process below, but now for pump leakage as dependent variable.
yleak = 'pump_leak'
aml_search_pump = H2OAutoML(max_runtime_secs = 600)
aml_search_pump.train(x = X, 
          y = yleak,
          training_frame = hf_train_extracted,
          leaderboard_frame = hf_test_extracted)
aml_search_pump.leaderboard